# Ames Housing - Neural Network
- Author: Oliver Mueller
- Last update: 26.01.2024

## Initialize notebook
Load required packages. Set up workspace, e.g., set theme for plotting and initialize the random number generator.

In [4]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor

In [5]:
plt.style.use('fivethirtyeight')

## Problem description

Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence. With 76 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this dataset challenges you to predict the final price of each home. More: <https://www.kaggle.com/c/house-prices-advanced-regression-techniques>


## Load data

Load training data from CSV file.

In [6]:
data = pd.read_csv('data/train.csv')

In [7]:
data.head()

,house_id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ThreeSsnPorch,ScreenPorch,PoolArea,Fence,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,20,RL,80,10400,Pave,none,Reg,Lvl,AllPub,...,0,0,0,MnPrv,0,6,2009,WD,Family,152000
1,2,60,RL,0,28698,Pave,none,IR2,Low,AllPub,...,0,225,0,none,0,6,2009,WD,Abnorml,185000
2,3,90,RL,70,9842,Pave,none,Reg,Lvl,AllPub,...,0,0,0,none,0,3,2007,WD,Normal,101800
3,4,90,RL,60,7200,Pave,none,Reg,Lvl,AllPub,...,0,0,0,none,0,6,2009,WD,Normal,90000
4,5,190,RM,63,7627,Pave,none,Reg,Lvl,AllPub,...,0,0,0,none,0,10,2009,WD,Normal,94550


## Prepare data

First, we will remove some columns that are not useful for our task.

In [8]:
data = data.drop(['house_id', 'YrSold', 'MoSold', 'SaleCondition', 'SaleType'], axis=1)

Next, we will split the data into features (*X*) and labels (*y*) and into training (*X_train, y_train*) and test (*X_test, y_test*) sets.

In [9]:
X = data.drop("SalePrice", axis=1)
y = data["SalePrice"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Finally, we will do some feature engineering. It is important to use only information from the training set for feature engineering, and the mechanistically repeat these steps on the test set.

Typically, feature engineering depends strongly on the datatype of the variables. Hence, we will first determine which variables are categorical and which are numerical. Subsequentally, we will transform these variables seperately.

In [10]:
categorical_features = X_train.select_dtypes(include='object').columns
numerical_features = X_train.select_dtypes(exclude='object').columns

The categorical variables must be transformed into numerical representations, e.g., by one-hot encdoing them.

In [11]:
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
enc.fit(X_train[categorical_features])

X_train_cat = enc.transform(X_train[categorical_features])
X_test_cat = enc.transform(X_test[categorical_features])

X_train_cat = pd.DataFrame(X_train_cat, columns=enc.get_feature_names_out(categorical_features))
X_test_cat = pd.DataFrame(X_test_cat, columns=enc.get_feature_names_out(categorical_features))

In [12]:
X_train_cat.head()

,MSZoning_A(agr),MSZoning_C(all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,...,GarageCond_TA,GarageCond_none,PavedDrive_N,PavedDrive_P,PavedDrive_Y,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_none
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


The numerical variables will be standardized, that is, we will subtract the mean and divide by the standard deviation. This is especially important for LASSO, as all coefficients need to be comparable in terms of units and magnitudes.

In [13]:
scaler = StandardScaler()
scaler.fit(X_train[numerical_features]) 

X_train_num = scaler.transform(X_train[numerical_features])
X_test_num = scaler.transform(X_test[numerical_features])

X_train_num = pd.DataFrame(X_train_num, columns=numerical_features)
X_test_num = pd.DataFrame(X_test_num, columns=numerical_features)

In [14]:
X_train_num.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,ThreeSsnPorch,ScreenPorch,PoolArea,MiscVal
0,2.394594,-0.998884,-0.964781,0.671861,-0.528618,1.101158,0.960866,-0.290751,-0.969935,-0.293955,...,-0.929768,0.341572,-0.411812,-0.747661,-0.116853,-0.363305,-0.106132,-0.296792,-0.072239,-0.090035
1,0.527998,-1.704399,1.210361,-0.043088,1.276772,-0.156698,-0.866729,-0.578245,-0.969935,-0.293955,...,0.610263,0.341572,-0.168654,3.923522,2.723148,-0.363305,-0.106132,-0.296792,17.507890,-0.090035
2,-0.871949,1.147058,0.210718,-0.758036,-0.528618,0.108114,-0.481972,-0.578245,0.496704,-0.293955,...,-0.929768,0.341572,-0.131245,-0.747661,-0.696446,-0.363305,-0.106132,-0.296792,-0.072239,-0.090035
3,1.461296,-1.704399,-0.895097,0.671861,-0.528618,1.101158,0.960866,-0.474747,-0.969935,-0.293955,...,0.610263,0.341572,-0.327642,0.400206,-0.406650,-0.363305,-0.106132,-0.296792,-0.072239,-0.090035
4,-0.871949,0.647318,-0.270732,-0.758036,-0.528618,-0.355307,-1.155297,-0.578245,1.458978,0.608922,...,2.150294,-0.989920,-0.547419,-0.747661,-0.696446,-0.363305,-0.106132,-0.296792,-0.072239,-0.090035


Let's fuse the enginnered categorical and numerical variables again.

In [15]:
X_train = pd.concat([X_train_num, X_train_cat], axis=1)
X_test = pd.concat([X_test_num, X_test_cat], axis=1)

In [16]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageCond_TA,GarageCond_none,PavedDrive_N,PavedDrive_P,PavedDrive_Y,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_none
0,2.394594,-0.998884,-0.964781,0.671861,-0.528618,1.101158,0.960866,-0.290751,-0.969935,-0.293955,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.527998,-1.704399,1.210361,-0.043088,1.276772,-0.156698,-0.866729,-0.578245,-0.969935,-0.293955,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,-0.871949,1.147058,0.210718,-0.758036,-0.528618,0.108114,-0.481972,-0.578245,0.496704,-0.293955,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,1.461296,-1.704399,-0.895097,0.671861,-0.528618,1.101158,0.960866,-0.474747,-0.969935,-0.293955,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,-0.871949,0.647318,-0.270732,-0.758036,-0.528618,-0.355307,-1.155297,-0.578245,1.458978,0.608922,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


## Neural Network

In the following, we will create a neural network with one hidden layer comprising 256 neurons. We will use the ReLU activation function and train for 100 epochs.

In [40]:
model_mlp = MLPRegressor(hidden_layer_sizes=(32,), activation = 'relu', learning_rate_init = 0.001, max_iter=10000, solver='adam', random_state=42)

In [41]:
model_mlp.fit(X_train, y_train)

MLPRegressor(hidden_layer_sizes=(32,), max_iter=10000, random_state=42)

In [42]:
# Training data
pred_train = model_mlp.predict(X_train)
r2_train = r2_score(y_train, pred_train)
rmse_train = root_mean_squared_error(y_train, pred_train)
print('R2 on training set:', round(r2_train, 2))
print('RMSE on training set:', round(rmse_train, 2))

print("===")

# Test data
pred_test = model_mlp.predict(X_test)
r2_test = r2_score(y_test, pred_test)
rmse_test = root_mean_squared_error(y_test, pred_test)
print('R2 on test set:', round(r2_test, 2))
print('RMSE on test set:', round(rmse_test, 2))

R2 on training set: 0.93
RMSE on training set: 20599.27
===
R2 on test set: 0.91
RMSE on test set: 21635.66
